# 📝 Google Forms Creator for SME Evaluation

**Simple Google Colab notebook to create forms for expert evaluation of AI models**

This notebook creates Google Forms based on your sample data for Subject Matter Expert (SME) evaluation.

## 🚀 Quick Start:
1. Upload your JSON data file to Colab
2. Update the `PROJECT_ID` and `JSON_FILE_PATH` below
3. Run all cells
4. Get your form URLs!


## 📦 Setup and Authentication


In [ ]:
# Install required packages
%pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [ ]:
import json
import os
from typing import List, Dict, Any, Optional
from google.colab import auth
from googleapiclient.discovery import build
from google.auth import default

print("📦 Imports successful!")


In [ ]:
# 🔧 CONFIGURATION - UPDATE THESE VALUES
PROJECT_ID = "xxxx"  # Your Google Cloud Project ID
JSON_FILE_PATH = "/content/sample_data_template.json"  # Path to your JSON data file
FORM_TITLE = "Outbreak Risk Assessment - Expert Evaluation"  # Your form title

print(f"🔧 Project ID: {PROJECT_ID}")
print(f"📂 Data file: {JSON_FILE_PATH}")
print(f"📝 Form title: {FORM_TITLE}")


In [ ]:
# 🔐 Authentication
print(f"🔧 Setting up project: {PROJECT_ID}")

# Set environment variables
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

# Authenticate
print("🔐 Authenticating...")
auth.authenticate_user()
creds, _ = default()

if hasattr(creds, 'with_quota_project'):
    creds = creds.with_quota_project(PROJECT_ID)

print(f"✅ Using project: {PROJECT_ID}")

# Build Forms service
forms_service = build('forms', 'v1', credentials=creds)
print("✅ Google Forms API connected successfully!")


## 📊 Load and Prepare Data


In [ ]:
# 📂 Load your JSON data
print(f"📂 Loading data from: {JSON_FILE_PATH}")

try:
    with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
    print(f"✅ Loaded {len(sample_data)} samples!")
    
    # Show first sample structure
    if sample_data:
        print("\n📋 Sample data structure:")
        first_sample = sample_data[0]
        for key in first_sample.keys():
            value = str(first_sample[key])[:100] + "..." if len(str(first_sample[key])) > 100 else first_sample[key]
            print(f"  • {key}: {value}")
            
except FileNotFoundError:
    print(f"❌ File not found: {JSON_FILE_PATH}")
    print("📤 Please upload your JSON file to Colab and update the path above")
    print("\n🔧 Creating sample data for demonstration...")
    
    # Create sample data structure for SME evaluation
    sample_data = [
        {
            "title": "Sample Article 1",
            "domain": "Infectious Disease",
            "factors_description": "Transmission risk, population density, containment measures",
            "article_content": "Health officials report increased cases of respiratory illness in urban areas. Local hospitals seeing surge in admissions...",
            "reference_score": 3,
            "reference_reasoning": "Moderate risk due to urban transmission but containment measures in place",
            "model_a_score": 2,
            "model_a_reasoning": "Low risk assessment based on current containment",
            "model_b_score": 4,
            "model_b_reasoning": "High risk due to urban density and hospital surge"
        },
        {
            "title": "Sample Article 2",
            "domain": "Zoonotic Disease",
            "factors_description": "Animal-human interface, surveillance systems, outbreak history",
            "article_content": "Veterinary authorities investigating unusual deaths in livestock. No human cases reported yet but monitoring continues...",
            "reference_score": 2,
            "reference_reasoning": "Low-moderate risk with good surveillance but animal deaths concerning",
            "model_a_score": 1,
            "model_a_reasoning": "Very low risk as no human cases detected",
            "model_b_score": 3,
            "model_b_reasoning": "Moderate risk due to potential zoonotic transmission"
        }
    ]
    print(f"✅ Created {len(sample_data)} sample entries for demonstration")

print(f"\n🎯 Ready to create form with {len(sample_data)} samples")


## 🚀 Create the Google Form


In [ ]:
# 🏗️ Create SME Evaluation Form
print("🏗️ Creating SME evaluation form...")

# Step 1: Create basic form
print("📝 Creating basic form...")
form_request = {"info": {"title": f"{FORM_TITLE} - {len(sample_data)} Samples"}}

form = forms_service.forms().create(body=form_request).execute()
form_id = form['formId']
print(f"✅ Form created! ID: {form_id}")

# Step 2: Add description
print("📝 Adding description...")
try:
    forms_service.forms().batchUpdate(
        formId=form_id,
        body={
            "requests": [{
                "updateFormInfo": {
                    "info": {
                        "title": f"{FORM_TITLE} - {len(sample_data)} Samples",
                        "description": f"Expert evaluation of AI-generated outbreak risk assessments. {len(sample_data)} samples to evaluate."
                    },
                    "updateMask": "description"
                }
            }]
        }
    ).execute()
    print("✅ Description added!")
except Exception as e:
    print(f"⚠️ Description error: {e}")

print(f"📊 Form setup complete! Ready to add {len(sample_data)} samples.")


In [ ]:
# 📝 Add Form Content - Simple Version
print("📝 Adding form content...")

all_requests = []
current_index = 0

# Add SME Email field
all_requests.append({
    "createItem": {
        "item": {
            "title": "Your Email (Required)",
            "description": "Please enter your email address for identification",
            "questionItem": {
                "question": {
                    "required": True,
                    "textQuestion": {"paragraph": False}
                }
            }
        },
        "location": {"index": current_index}
    }
})
current_index += 1

# Add instructions
instructions = """You are evaluating AI-generated outbreak risk assessments for news articles.

**EVALUATION TASKS:**
1. **Reference Validation:** Rate the accuracy of our "gold standard" AI assessment
2. **Model Comparison:** Compare which of two AI models gives better risk assessment
3. **Expert Assessment:** Optionally provide your own expert opinion

**Risk Scores:** 1=Very Low, 2=Low, 3=Moderate, 4=High, 5=Very High

Rate based on scientific accuracy, appropriateness of score, and quality of reasoning."""

all_requests.append({
    "createItem": {
        "item": {
            "title": "📋 Instructions",
            "description": instructions,
            "textItem": {}
        },
        "location": {"index": current_index}
    }
})
current_index += 1

# Add questions for each sample
for i, sample in enumerate(sample_data):
    sample_num = i + 1
    
    # Sample header
    header = f"""═══════════════════════════════════════════════
SAMPLE {sample_num}: {sample.get('title', 'Sample Article')}
Domain: {sample.get('domain', 'Unknown Domain')}
═══════════════════════════════════════════════"""
    
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"📰 Sample {sample_num} - Article",
                "description": f"{header}\n\nArticle: {sample.get('article_content', 'Article content')[:500]}{'...' if len(sample.get('article_content', '')) > 500 else ''}",
                "textItem": {}
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1
    
    # Reference assessment display
    ref_text = f"""REFERENCE ASSESSMENT (GPT-O1):
Score: {sample.get('reference_score', 'X')}/5
Reasoning: {sample.get('reference_reasoning', 'Reference reasoning')}"""
    
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"🔍 Reference Assessment - Sample {sample_num}",
                "description": ref_text,
                "textItem": {}
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1
    
    # Reference quality question
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"1. Reference Quality - Sample {sample_num} *",
                "description": "Rate the REFERENCE assessment quality:",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "Poor (Major errors)"},
                                {"value": "Average (Acceptable)"},
                                {"value": "Good (Accurate)"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1
    
    # Model comparison display
    models_text = f"""MODEL COMPARISON:

MODEL A: Score {sample.get('model_a_score', 'Y')}/5
Reasoning: {sample.get('model_a_reasoning', 'Model A reasoning')}

MODEL B: Score {sample.get('model_b_score', 'Z')}/5  
Reasoning: {sample.get('model_b_reasoning', 'Model B reasoning')}"""
    
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"⚖️ Model Comparison - Sample {sample_num}",
                "description": models_text,
                "textItem": {}
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1
    
    # Model comparison question
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"2. Which model is better? - Sample {sample_num} *",
                "description": "Which model gives MORE ACCURATE risk assessment?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "Model A is clearly better"},
                                {"value": "Model A is slightly better"},
                                {"value": "Both are equivalent"},
                                {"value": "Model B is slightly better"},
                                {"value": "Model B is clearly better"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1
    
    # Expert score (optional)
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"3. Your Expert Score (Optional) - Sample {sample_num}",
                "description": "What risk score would YOU assign (1-5)?",
                "questionItem": {
                    "question": {
                        "required": False,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "1 - Very low risk"},
                                {"value": "2 - Low risk"},
                                {"value": "3 - Moderate risk"},
                                {"value": "4 - High risk"},
                                {"value": "5 - Very high risk"}
                            ]
                        }
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1
    
    # Comments
    all_requests.append({
        "createItem": {
            "item": {
                "title": f"4. Comments (Optional) - Sample {sample_num}",
                "description": "Any additional comments or concerns?",
                "questionItem": {
                    "question": {
                        "required": False,
                        "textQuestion": {"paragraph": True}
                    }
                }
            },
            "location": {"index": current_index}
        }
    })
    current_index += 1

print(f"📊 Prepared {len(all_requests)} form items")


In [ ]:
# 🔧 Add Items to Form in Batches
print("📝 Adding items to form...")

batch_size = 5  # Small batches to avoid API limits
success_count = 0

for i in range(0, len(all_requests), batch_size):
    batch = all_requests[i:i+batch_size]
    
    try:
        forms_service.forms().batchUpdate(
            formId=form_id,
            body={"requests": batch}
        ).execute()
        success_count += len(batch)
        print(f"✅ Added {len(batch)} items (total: {success_count}/{len(all_requests)})")
        
    except Exception as e:
        print(f"❌ Batch error: {e}")
        # Try individually
        for j, request in enumerate(batch):
            try:
                forms_service.forms().batchUpdate(
                    formId=form_id,
                    body={"requests": [request]}
                ).execute()
                success_count += 1
                print(f"✅ Added item individually ({success_count}/{len(all_requests)})")
            except Exception as eq:
                print(f"❌ Individual error {i+j+1}: {eq}")

print(f"📊 Successfully added {success_count}/{len(all_requests)} items")

# Generate URLs - CORRECTED FORMAT
edit_url = f"https://docs.google.com/forms/d/{form_id}/edit"
view_url = f"https://docs.google.com/forms/d/{form_id}/viewform"  # Remove the /e/ part
responses_url = f"https://docs.google.com/forms/d/{form_id}/edit#responses"  # Responses tab

print(f"\n🎉 FORM CREATED SUCCESSFULLY!")
print(f"📝 Edit Form: {edit_url}")
print(f"👁️ View Form: {view_url}")
print(f"📊 View Responses: {responses_url}")

# Let's also test the form access to make sure it's working
print(f"\n🔍 Testing form access...")
try:
    form_data = forms_service.forms().get(formId=form_id).execute()
    print(f"✅ Form is accessible!")
    print(f"   Title: {form_data['info']['title']}")
    print(f"   Items: {len(form_data.get('items', []))} questions/sections")
    
    # Let's also get the published URL from the form data
    if 'responderUri' in form_data:
        public_url = form_data['responderUri']
        print(f"📋 Public Form URL: {public_url}")
    else:
        print("📋 Public URL will be available once form is published")
        
except Exception as e:
    print(f"❌ Form access test failed: {e}")


## 📋 Form Information and Results


In [ ]:
# 💾 Save Form Information - Get correct URLs from API
print("🔍 Getting correct URLs from Google Forms API...")

try:
    # Get the form data to extract the correct URLs
    form_data = forms_service.forms().get(formId=form_id).execute()
    
    # Extract the correct URLs
    edit_url = f"https://docs.google.com/forms/d/{form_id}/edit"
    
    # The responderUri gives us the correct public URL
    if 'responderUri' in form_data:
        view_url = form_data['responderUri']
        print(f"✅ Got public URL from API: {view_url}")
    else:
        # Fallback URL format
        view_url = f"https://docs.google.com/forms/d/{form_id}/viewform"
        print(f"⚠️ Using fallback URL format: {view_url}")
    
    # Responses URL - this goes to the responses tab of the edit page
    responses_url = f"https://docs.google.com/forms/d/{form_id}/edit#responses"
    
    print(f"✅ All URLs generated successfully!")
    
except Exception as e:
    print(f"⚠️ Could not get URLs from API: {e}")
    # Use fallback URLs
    edit_url = f"https://docs.google.com/forms/d/{form_id}/edit"
    view_url = f"https://docs.google.com/forms/d/{form_id}/viewform"
    responses_url = f"https://docs.google.com/forms/d/{form_id}/edit#responses"

form_info = {
    'form_id': form_id,
    'edit_url': edit_url,
    'view_url': view_url,
    'responses_url': responses_url,
    'total_samples': len(sample_data),
    'json_file_used': JSON_FILE_PATH,
    'project_id': PROJECT_ID,
    'form_title': FORM_TITLE
}

# Save to file
with open('/content/form_results.json', 'w') as f:
    json.dump(form_info, f, indent=2)

print(f"💾 Form information saved to /content/form_results.json")
print(f"📊 Total samples processed: {len(sample_data)}")
print(f"🎯 Form ready for SME evaluation!")

# Display the information nicely
print("\n" + "="*60)
print("📋 FORM SUMMARY")
print("="*60)
print(f"📝 Form Title: {FORM_TITLE}")
print(f"🆔 Form ID: {form_id}")
print(f"📊 Total Samples: {len(sample_data)}")
print(f"🔧 Project ID: {PROJECT_ID}")
print("\n🔗 IMPORTANT LINKS:")
print(f"✏️  Edit Form: {edit_url}")
print(f"👀 View Form: {view_url}")
print(f"📈 Responses: {responses_url}")
print("\n💡 NEXT STEPS:")
print("1. Click the 'View Form' link to test your form")
print("2. Share the 'View Form' link with your SME evaluators")
print("3. Monitor responses using the 'Responses' link")
print("4. Download form_results.json for your records")


## 🔗 URL Troubleshooting

**If the View URL doesn't work immediately:**

1. **Wait a few minutes** - Google Forms sometimes takes time to make forms publicly accessible
2. **Try the Edit URL first** - Click "Preview" button in the top-right to test the form
3. **Check form settings** - Make sure the form accepts responses

**Alternative ways to get the correct View URL:**
- Open the **Edit URL** → Click **Send** button → Copy the link from there
- Open the **Edit URL** → Click the **Preview** eye icon → Copy URL from browser
- Replace `YOUR_FORM_ID` in: `https://docs.google.com/forms/d/YOUR_FORM_ID/viewform`

**For Responses:**
- Use the **Edit URL** → Click **Responses** tab
- Direct link: `https://docs.google.com/forms/d/YOUR_FORM_ID/edit#responses`

**Common Issues:**
- **Form not published**: New forms might need a few minutes to become publicly accessible
- **Permissions**: Make sure the form is set to accept responses from anyone
- **URL format**: Different Google accounts might use slightly different URL formats


## 📖 Instructions for Using Your Own Data

To use this notebook with your own data:

### 1. 📤 Upload Your JSON File
- Click the folder icon in the left sidebar of Colab
- Upload your JSON data file
- Update the `JSON_FILE_PATH` variable in the configuration cell above

### 2. 📊 Required JSON Structure
Your JSON file should contain an array of objects with these fields:
```json
[
  {
    "title": "Article title",
    "domain": "Risk domain (e.g., Infectious Disease)",
    "factors_description": "Key risk factors for this domain",
    "article_content": "Full article text",
    "reference_score": 3,
    "reference_reasoning": "GPT-o1 reasoning",
    "model_a_score": 2,
    "model_a_reasoning": "Model A reasoning",
    "model_b_score": 4,
    "model_b_reasoning": "Model B reasoning"
  }
]
```

### 3. 🔧 Configuration
- Update `PROJECT_ID` with your Google Cloud Project ID
- Update `FORM_TITLE` with your desired form title
- Run all cells again

### 4. 🚀 That's it!
Your custom form will be created with your data.

---

**💡 Tips:**
- The notebook will create sample data if your JSON file is not found
- You can modify the questions by editing the form creation cells
- All form URLs are saved to `/content/form_results.json`
- You can download the notebook and run it locally if needed
